## Network Distances Through Namma Metro to my Favorite Resaurants in Bengaluru

This assignment investigates accessibility in Bengaluru’s Namma Metro by comparing straight-line (Euclidean) distances with network-constrained travel distances. Using a selection of my favorite restaurants across the city as destination points, the analysis examines how apparent proximity differs from actual travel effort on the metro network.

### Tools and Libraries — imports and setup

- geopandas: read and manipulate GeoJSON data for metro lines, stations, and restaurants.
- shapely: geometric operations for Euclidean distances and spatial relationships.
- networkx: build and analyze the metro network graph and compute shortest paths.
- pandas: tidy tabular data, joins, and summaries.
- folium or kepler.gl: quick interactive maps to visualize routes and accessibility.
- numpy: efficient numeric operations during distance calculations.

In [16]:
# Core data and geometry libraries
import pandas as pd
import numpy as np
import geopandas as gpd

# Geometry operations (Euclidean distances, spatial relationships)
import shapely
from shapely.geometry import Point, LineString, shape

# Network analysis (build metro graph, shortest paths)
import networkx as nx

# Mapping (interactive)
import folium

# Optional: kepler.gl (if installed)
try:
    from keplergl import KeplerGl

    KEPLER_AVAILABLE = True
except ImportError:
    KEPLER_AVAILABLE = False

# Quick confirmation (kept silent unless needed)
# print("Imports loaded. kepler.gl available:", KEPLER_AVAILABLE)

### Load restaurants dataset

- What this does: Reads `blr-restaurants.geojson` into a GeoDataFrame as point features.
- Outputs: Prints feature count and CRS, and previews the first few rows.
- How it works: Uses GeoPandas `read_file` to load GeoJSON, keeping geometries and attributes for later mapping and selection.

In [17]:
# Load restaurants (points)
restaurants_path = "blr-restaurants.geojson"
restaurants_gdf = gpd.read_file(restaurants_path)

print(f"Restaurants loaded: {len(restaurants_gdf)} features")
print("CRS:", restaurants_gdf.crs)
restaurants_gdf.head(2)

Restaurants loaded: 24 features
CRS: EPSG:4326


,name,category,cuisine,neighborhood,geometry
0,Mavalli Tiffin Rooms (MTR),restaurant,South Indian,Lalbagh Road,POINT (77.58555 12.9552)
1,Sri Sagar (CTR),restaurant,South Indian,Malleshwaram,POINT (77.56953 12.99811)


### Load home location

- What this does: Reads `home.geojson` as a single point (your home).
- Outputs: Prints feature count and CRS, and previews the record.
- How it works: Loads the GeoJSON into a GeoDataFrame; the geometry is used as the route starting point.

In [18]:
# Load home (single point)
home_path = "home.geojson"
home_gdf = gpd.read_file(home_path)

print(f"Home features loaded: {len(home_gdf)}")
print("CRS:", home_gdf.crs)
home_gdf.head(2)

Home features loaded: 1
CRS: EPSG:4326


,type,neighborhood,geometry
0,home,Doddakalasandra,POINT (77.55594 12.88171)


### Load and split metro network

- What this does: Reads `metro_lines_styled_cgpt.geojson`, filters empty geometries, and splits features into line segments and station points.
- Outputs: Prints counts and CRS for lines and stations, plus a preview of each.
- How it works: Uses geometry types to separate LineString/MultiLineString (lines) from Point/MultiPoint (stations); explodes MultiPoint collections into individual stations.

In [19]:
# Load Namma Metro lines and stations from a single GeoJSON
metro_path = "metro_lines_styled_cgpt.geojson"

metro_all_gdf = gpd.read_file(metro_path)
# Filter out empty/null geometries
metro_all_gdf = metro_all_gdf[
    metro_all_gdf.geometry.notnull() & ~metro_all_gdf.geometry.is_empty
].copy()

# Split by geometry type
geom_type = metro_all_gdf.geometry.geom_type
metro_lines_gdf = metro_all_gdf[
    geom_type.isin(["LineString", "MultiLineString"])
].copy()
metro_stations_gdf = metro_all_gdf[geom_type.isin(["Point", "MultiPoint"])].copy()

# Explode MultiPoint to individual Point features if present
if (
    not metro_stations_gdf.empty
    and (metro_stations_gdf.geometry.geom_type == "MultiPoint").any()
):
    metro_stations_gdf = metro_stations_gdf.explode(
        index_parts=False, ignore_index=True
    )

print(f"Metro lines loaded: {len(metro_lines_gdf)} features")
print("Lines CRS:", metro_lines_gdf.crs)
print(f"Metro stations loaded: {len(metro_stations_gdf)} features")
print("Stations CRS:", metro_stations_gdf.crs)

# Peek at the first features
metro_lines_gdf.head(2), metro_stations_gdf.head(2)

Metro lines loaded: 10 features
Lines CRS: EPSG:4326
Metro stations loaded: 120 features
Stations CRS: EPSG:4326


(                                                Name description timestamp  \
 0  Line-1 (Purple): Mysore Road - Baiyappanahalli...      purple      None   
 1  Line-2 (Green): Hessarghatta Cross - Yelachena...       green      None   
 
   begin   end altitudeMode  tessellate  extrude  visibility drawOrder  icon  \
 0  None  None         None           1        0          -1      None  None   
 1  None  None         None           1        0          -1      None  None   
 
     stroke  stroke-width  stroke-opacity marker-color marker-size  \
 0  #800080           4.0             1.0         None        None   
 1  #008000           4.0             1.0         None        None   
 
   marker-symbol                                           geometry  
 0          None  LINESTRING Z (77.65785 12.99363 0, 77.65668 12...  
 1          None  LINESTRING Z (77.49549 13.04979 0, 77.49955 13...  ,
                                Name description timestamp begin   end  \
 10  Hesaraghatta Cros

### Let's go to Bengaluru and visit my favorite restaurants!

- What this does: Creates a base Folium map centered on Bengaluru and adds two layers: restaurants and home.
- Outputs: An interactive map with toggleable layers (Layer Control) showing markers for restaurants and home.

In [20]:
# Stage 1: Restaurants + Home
from IPython.display import display

blr_center = [12.9716, 77.5946]

m1 = folium.Map(
    location=blr_center, zoom_start=12, tiles="CartoDB positron", control_scale=True
)

# Restaurants layer (use pin icon instead of circle)
restaurants_fg = folium.FeatureGroup(name="Restaurants", show=True)
for _, row in restaurants_gdf.iterrows():
    geom = row.geometry
    if geom and geom.geom_type == "Point":
        folium.Marker(
            location=[geom.y, geom.x],
            icon=folium.Icon(color="red", icon="map-marker", prefix="fa"),
            tooltip=str(row.get("name", "Restaurant")),
        ).add_to(restaurants_fg)
restaurants_fg.add_to(m1)

# Home layer
home_fg = folium.FeatureGroup(name="Home", show=True)
for _, row in home_gdf.iterrows():
    geom = row.geometry
    if geom and geom.geom_type == "Point":
        folium.Marker(
            location=[geom.y, geom.x],
            icon=folium.Icon(color="blue", icon="home", prefix="fa"),
            tooltip=str(row.get("name", "Home")),
        ).add_to(home_fg)
home_fg.add_to(m1)

folium.LayerControl(position="topright").add_to(m1)

display(m1)

### Mapping utilities and styling helpers

- What this does: Defines reusable functions to create base maps, compute centers, and add layers for home, destination, restaurants, metro lines, and stations.
- Outputs: No direct output; functions are used by later map-building cells.
- How it works: Encapsulates Folium layer creation and color normalization, keeping downstream cells concise and consistent.

In [21]:
# Map utilities: shared styling and helper functions to avoid repetition
import folium
import geopandas as gpd
import pandas as pd
import re
from typing import Optional, Tuple

# Defaults
BLR_CENTER = [12.9716, 77.5946]
DEFAULT_LINE_COLOR = "#1f77b4"
RESTAURANT_ICON = dict(color="red", icon="map-marker", prefix="fa")
HOME_ICON = dict(color="blue", icon="home", prefix="fa")
DEST_ICON = RESTAURANT_ICON
STATION_STYLE = dict(
    radius=4, color="#000", fill=True, fill_color="#fff", fill_opacity=0.9
)

# Potential color keys present in GeoJSON properties
COLOR_KEYS = [
    "stroke",
    "color",
    "line_color",
    "lineColour",
    "line_color_hex",
    "style_color",
    "strokeColor",
    "linecolour",
    "Colour",
    "Color",
]


def rgb_to_hex(s: str) -> Optional[str]:
    try:
        nums = [int(x) for x in re.findall(r"\d+", s)[:3]]
        if len(nums) == 3:
            return "#%02x%02x%02x" % tuple(max(0, min(255, n)) for n in nums)
    except Exception:
        return None
    return None


def normalize_color(val) -> Optional[str]:
    if val is None:
        return None
    if not isinstance(val, str):
        try:
            val = str(val)
        except Exception:
            return None
    s = val.strip()
    if not s:
        return None
    if s.startswith("#") and len(s) in (4, 7):
        return s
    if s.lower().startswith("rgb"):
        hx = rgb_to_hex(s)
        return hx or None
    return s


def color_from_props(props: dict) -> str:
    # Direct keys
    for k in COLOR_KEYS:
        if k in props and pd.notna(props[k]):
            nc = normalize_color(props[k])
            if nc:
                return nc
    # Nested 'style' dict
    style_prop = props.get("style") if isinstance(props, dict) else None
    if isinstance(style_prop, dict):
        for k in ("color", "stroke"):
            nc = normalize_color(style_prop.get(k))
            if nc:
                return nc
    return DEFAULT_LINE_COLOR


def create_base_map(center: list = BLR_CENTER, zoom_start: int = 12) -> folium.Map:
    return folium.Map(
        location=center,
        zoom_start=zoom_start,
        tiles="CartoDB positron",
        control_scale=True,
    )


def compute_center(
    start_pt=None, end_pt=None, default_center: list = BLR_CENTER
) -> list:
    latlon = []
    try:
        if start_pt is not None:
            latlon.append((start_pt.y, start_pt.x))
    except Exception:
        pass
    try:
        if end_pt is not None:
            latlon.append((end_pt.y, end_pt.x))
    except Exception:
        pass
    if len(latlon) == 2:
        return [
            (latlon[0][0] + latlon[1][0]) / 2.0,
            (latlon[0][1] + latlon[1][1]) / 2.0,
        ]
    if len(latlon) == 1:
        return [latlon[0][0], latlon[0][1]]
    return default_center


def add_home_layer(
    m: folium.Map, home_gdf: gpd.GeoDataFrame, name: str = "Home"
) -> folium.FeatureGroup:
    fg = folium.FeatureGroup(name=name, show=True)
    try:
        if home_gdf is not None and len(home_gdf) > 0:
            for _, row in home_gdf.iterrows():
                geom = row.geometry
                if geom is not None and geom.geom_type == "Point":
                    folium.Marker(
                        location=[geom.y, geom.x],
                        icon=folium.Icon(**HOME_ICON),
                        tooltip=str(row.get("name", name)),
                    ).add_to(fg)
    except Exception:
        pass
    fg.add_to(m)
    return fg


def add_destination_layer(
    m: folium.Map, point, label: str = "Destination"
) -> folium.FeatureGroup:
    fg = folium.FeatureGroup(name=label, show=True)
    if point is not None:
        folium.Marker(
            location=[point.y, point.x],
            icon=folium.Icon(**DEST_ICON),
            tooltip=str(label),
        ).add_to(fg)
    fg.add_to(m)
    return fg


def add_restaurants_layer(
    m: folium.Map, restaurants_gdf: gpd.GeoDataFrame, name: str = "Restaurants"
) -> folium.FeatureGroup:
    fg = folium.FeatureGroup(name=name, show=True)
    try:
        if restaurants_gdf is not None and len(restaurants_gdf) > 0:
            for _, row in restaurants_gdf.iterrows():
                geom = row.geometry
                if geom is not None and geom.geom_type == "Point":
                    folium.Marker(
                        location=[geom.y, geom.x],
                        icon=folium.Icon(**RESTAURANT_ICON),
                        tooltip=str(row.get("name", "Restaurant")),
                    ).add_to(fg)
    except Exception:
        pass
    fg.add_to(m)
    return fg


def add_metro_lines_layer(
    m: folium.Map, lines_gdf: gpd.GeoDataFrame, name: str = "Metro Lines"
) -> folium.FeatureGroup:
    fg = folium.FeatureGroup(name=name, show=True)
    try:
        if lines_gdf is not None and len(lines_gdf) > 0:
            style_fn = lambda feature: {
                "color": color_from_props(feature.get("properties", {})),
                "weight": 3,
                "opacity": 0.9,
            }
            tooltip = None
            if "line" in lines_gdf.columns:
                try:
                    tooltip = folium.GeoJsonTooltip(fields=["line"], aliases=["Line:"])
                except Exception:
                    tooltip = None
            folium.GeoJson(
                data=lines_gdf.to_json(),
                name=name,
                style_function=style_fn,
                tooltip=tooltip,
            ).add_to(fg)
    except Exception as e:
        print(f"Could not add metro lines: {e}")
    fg.add_to(m)
    return fg


def add_metro_stations_layer(
    m: folium.Map, stations_gdf: gpd.GeoDataFrame, name: str = "Metro Stations"
) -> folium.FeatureGroup:
    fg = folium.FeatureGroup(name=name, show=True)
    try:
        if stations_gdf is not None and len(stations_gdf) > 0:
            for _, row in stations_gdf.iterrows():
                geom = row.geometry
                if geom is not None and geom.geom_type == "Point":
                    folium.CircleMarker(
                        location=[geom.y, geom.x],
                        tooltip=str(row.get("name", "Station")),
                        **STATION_STYLE,
                    ).add_to(fg)
    except Exception as e:
        print(f"Could not add metro stations: {e}")
    fg.add_to(m)
    return fg


def add_layer_control(m: folium.Map):
    folium.LayerControl(position="topright").add_to(m)
    return m

### Let’s put it all on the map

We’ll add the datasets to an interactive map in this order:
1) Restaurants and Home (points)
2) Metro Lines (network)
3) Metro Stations (points)

Use the layer control to toggle visibility and explore.

In [22]:
# Stage 2: Restaurants + Home + Metro Lines + Metro Stations (using utilities)
from IPython.display import display

center = BLR_CENTER
m2 = create_base_map(center=center, zoom_start=12)

add_restaurants_layer(m2, restaurants_gdf, name="Restaurants")
add_home_layer(m2, home_gdf, name="Home")
add_metro_lines_layer(m2, metro_lines_gdf, name="Metro Lines")
add_metro_stations_layer(m2, metro_stations_gdf, name="Metro Stations")

add_layer_control(m2)

display(m2)

### Which restaurant shall I take you to?

Pick a destination from the dropdown below. Your choice will be saved as the endpoint (Home is the start point).

In [23]:
# Destination picker: dropdown of all known places
from IPython.display import display, HTML
import re

# Try to import ipywidgets; fall back gracefully if unavailable
widgets_available = True
try:
    import ipywidgets as widgets
except Exception as _e:
    widgets_available = False

# Collect candidate names from restaurants (and mall-like names if present)
name_cols = ["name", "Name", "title", "Title", "place", "Place", "label", "Label"]


def extract_names(gdf):
    if gdf is None or len(gdf) == 0:
        return []
    # Use the first name-like column found
    for col in name_cols:
        if col in gdf.columns:
            vals = gdf[col].dropna().astype(str).str.strip()
            return [v for v in vals if v]
    # Fallback to stringified index if no obvious name columns
    return [str(i) for i in range(len(gdf))]


# Restaurants names
restaurant_names = sorted(set(extract_names(restaurants_gdf)))
# Heuristic: include names that look like malls (substring match)
mall_like = [n for n in restaurant_names if "mall" in n.lower()]
all_places = sorted(set(restaurant_names + mall_like))

# Variables to expose for downstream cells
endpoint_name = None  # Chosen destination name (string)
endpoint_point = None  # Shapely Point of the chosen destination (if found)
start_point = None  # Home point geometry (start)

# Compute start_point from home_gdf if available
try:
    if (
        "home_gdf" in globals()
        and len(home_gdf) > 0
        and home_gdf.geometry.notnull().any()
    ):
        start_point = home_gdf.geometry.iloc[0]
except Exception:
    start_point = None


# Helper to find geometry by name in restaurants_gdf
from shapely.geometry import Point as _Point


def find_place_geometry_by_name(gdf, name_str):
    if gdf is None or len(gdf) == 0 or not name_str:
        return None
    target = str(name_str).strip().casefold()
    # Try exact, case-insensitive match in any known name column
    for col in name_cols:
        if col in gdf.columns:
            try:
                series = gdf[col].astype(str).str.strip().str.casefold()
                mask = series == target
                if mask.any():
                    geom = gdf.loc[mask, "geometry"].dropna()
                    if not geom.empty:
                        return geom.iloc[0]
            except Exception:
                pass
    # Fallback: contains match
    for col in name_cols:
        if col in gdf.columns:
            try:
                series = gdf[col].astype(str).str.strip().str.casefold()
                mask = series.str.contains(re.escape(target), na=False)
                if mask.any():
                    geom = gdf.loc[mask, "geometry"].dropna()
                    if not geom.empty:
                        return geom.iloc[0]
            except Exception:
                pass
    return None


if not widgets_available:
    # Graceful fallback: no interactive UI; guide the user
    print(
        "ipywidgets is not installed. To use the picker, install it and re-run this cell:"
    )
    print("  pip install ipywidgets")
    # Provide a safe default if possible
    if restaurant_names:
        endpoint_name = restaurant_names[0]
        endpoint_point = find_place_geometry_by_name(restaurants_gdf, endpoint_name)
        print(f"Defaulting endpoint_name to: {endpoint_name}")
    else:
        print("No places available to default.")
else:
    # Build the UI with only a dropdown
    dropdown = widgets.Dropdown(
        options=[("— Pick a destination —", "")] + [(n, n) for n in all_places],
        value="",
        description="Pick:",
        layout=widgets.Layout(width="60%"),
    )

    status = widgets.HTML(
        "<em>Select a destination from the dropdown, then click Confirm.</em>"
    )

    confirm_btn = widgets.Button(description="Confirm", button_style="primary")

    def on_confirm(_):
        global endpoint_name, endpoint_point
        chosen = dropdown.value.strip() if isinstance(dropdown.value, str) else ""
        if not chosen:
            status.value = "<span style='color:#b00'>Please select a destination from the dropdown.</span>"
            return
        endpoint_name = chosen
        endpoint_point = find_place_geometry_by_name(restaurants_gdf, endpoint_name)
        if endpoint_point is not None:
            status.value = (
                f"<b>Endpoint set:</b> {endpoint_name}. "
                f"<span style='color:#555'>(Home is the start point)</span>"
            )
        else:
            status.value = (
                f"<b>Endpoint set (name only):</b> {endpoint_name}. "
                f"<span style='color:#b00'>No geometry found — check the name spelling.</span>"
            )

    confirm_btn.on_click(on_confirm)

    ui = widgets.VBox(
        [
            widgets.HBox([dropdown]),
            widgets.HBox([confirm_btn]),
            status,
        ]
    )

    display(ui)

### Destination picker (dropdown)

- What this does: Lets you choose a restaurant from a dropdown. Sets `endpoint_name` and `endpoint_point`. Also computes `start_point` from `home_gdf`.
- Outputs: An interactive widget (or a safe default if ipywidgets is unavailable) and a status message when confirmed.
- How it works: Reads candidate names from the restaurants data, matches the selected name to a geometry, and stores it for routing cells.

### Nearest stations and walking legs

- What this does: Finds the nearest metro station to your home and to the chosen restaurant, draws dotted walking connectors, and shows a walk-distance breakdown and total.
- Outputs: Prints two walk legs (Home → first station, last station → destination) and the total walking distance; displays an interactive map highlighting nearest stations and walk legs.
- How it works: Uses a haversine function to compute distances on geographic coordinates and iterates over stations to find the nearest points.

In [27]:
# Map: Home + Destination + Nearest Stations (one step)
from IPython.display import display

center = compute_center(start_point, endpoint_point, default_center=BLR_CENTER)
m_dest = create_base_map(center=center, zoom_start=12)

# Core layers
add_home_layer(m_dest, home_gdf, name="Home")
add_destination_layer(
    m_dest, endpoint_point, label=globals().get("endpoint_name", "Destination")
)
add_metro_lines_layer(m_dest, metro_lines_gdf, name="Metro Lines")
add_metro_stations_layer(m_dest, metro_stations_gdf, name="Metro Stations")

# Also find and highlight the nearest metro stations to home and destination, and draw dotted walk legs
import math

R = 6371000.0  # meters


def haversine_xy(x1, y1, x2, y2):
    lon1, lat1, lon2, lat2 = map(math.radians, [x1, y1, x2, y2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


def nearest_station_point(stations_gdf, pt):
    if pt is None or stations_gdf is None or len(stations_gdf) == 0:
        return None
    best = None
    best_d = float("inf")
    for _, row in stations_gdf.iterrows():
        g = row.geometry
        if g is None or g.is_empty or g.geom_type != "Point":
            continue
        d = haversine_xy(pt.x, pt.y, g.x, g.y)
        if d < best_d:
            best_d = d
            best = g
    return best


start_station = (
    nearest_station_point(metro_stations_gdf, start_point)
    if "start_point" in globals()
    else None
)
end_station = (
    nearest_station_point(metro_stations_gdf, endpoint_point)
    if "endpoint_point" in globals()
    else None
)

# Highlights for nearest start/end stations
highlight_fg = folium.FeatureGroup(name="Nearest Stations (highlight)", show=True)
if start_station is not None:
    folium.CircleMarker(
        [start_station.y, start_station.x],
        radius=8,
        color="#2ecc71",
        fill=True,
        fill_color="#2ecc71",
        fill_opacity=0.95,
        tooltip="Start Station",
    ).add_to(highlight_fg)
if end_station is not None:
    folium.CircleMarker(
        [end_station.y, end_station.x],
        radius=8,
        color="#e74c3c",
        fill=True,
        fill_color="#e74c3c",
        fill_opacity=0.95,
        tooltip="End Station",
    ).add_to(highlight_fg)
highlight_fg.add_to(m_dest)

# Dotted connectors (walk legs)
walk_fg = folium.FeatureGroup(name="Walk (dotted)", show=True)
if start_point is not None and start_station is not None:
    folium.PolyLine(
        [[start_point.y, start_point.x], [start_station.y, start_station.x]],
        color="#666",
        weight=3,
        dash_array="6,8",
    ).add_to(walk_fg)
if endpoint_point is not None and end_station is not None:
    folium.PolyLine(
        [[end_station.y, end_station.x], [endpoint_point.y, endpoint_point.x]],
        color="#666",
        weight=3,
        dash_array="6,8",
    ).add_to(walk_fg)
walk_fg.add_to(m_dest)

# Compute and print walking distance breakdown and total (before showing the map)
walk_start_m = None  # Home -> first station
walk_end_m = None  # Last station -> destination
try:
    if start_point is not None and start_station is not None:
        walk_start_m = haversine_xy(
            start_point.x, start_point.y, start_station.x, start_station.y
        )
except Exception:
    walk_start_m = None
try:
    if endpoint_point is not None and end_station is not None:
        walk_end_m = haversine_xy(
            endpoint_point.x, endpoint_point.y, end_station.x, end_station.y
        )
except Exception:
    walk_end_m = None

total_walk_m = (walk_start_m or 0.0) + (walk_end_m or 0.0)

if walk_start_m is not None:
    print(
        f"Walk: Home → first station: {walk_start_m:.0f} m ({walk_start_m/1000:.2f} km)"
    )
else:
    print("Walk: Home → first station: n/a")
if walk_end_m is not None:
    print(
        f"Walk: last station → destination: {walk_end_m:.0f} m ({walk_end_m/1000:.2f} km)"
    )
else:
    print("Walk: last station → destination: n/a")
print(f"Total walking distance: {total_walk_m:.0f} m ({total_walk_m/1000:.2f} km)")

# Fit bounds (home, destination, and nearest stations if available)
bounds = []
for p in (start_point, endpoint_point, start_station, end_station):
    try:
        if p is not None:
            bounds.append([p.y, p.x])
    except Exception:
        pass
if bounds:
    try:
        m_dest.fit_bounds(bounds)
    except Exception:
        pass

add_layer_control(m_dest)

display(m_dest)

Walk: Home → first station: 480 m (0.48 km)
Walk: last station → destination: 1792 m (1.79 km)
Total walking distance: 2272 m (2.27 km)


### Visualize the metro network (nodes + edges)

- What this does: Loads a combined nodes+edges GeoJSON (or separate files) and plots the network on an interactive map.
- Outputs: An interactive map with edges and nodes layers.
- How it works: Splits features by geometry type, computes bounds to set the view, and draws edges via GeoJson and nodes as circle markers.

In [28]:
# Plot a custom network from nodes/edges GeoJSON on a map
import geopandas as gpd
import folium
import os

# Configure one of the following:
# 1) A single combined GeoJSON containing Points (nodes) and LineStrings (edges)
combined_path = "metro_network_nodes_edges.geojson"
# 2) Or separate files for nodes and edges
nodes_path = "nodes.geojson"
edges_path = "edges.geojson"

nodes_gdf = None
edges_gdf = None

# Try combined file first
try:
    if os.path.exists(combined_path):
        combo = gpd.read_file(combined_path)
        combo = combo[combo.geometry.notnull() & ~combo.geometry.is_empty].copy()
        geom_type = combo.geometry.geom_type
        nodes_gdf = combo[geom_type == "Point"].copy()
        edges_gdf = combo[geom_type.isin(["LineString", "MultiLineString"])].copy()
except Exception as e:
    print(f"Could not read combined file: {e}")

# Fallback to separate files
if (nodes_gdf is None or nodes_gdf.empty) and os.path.exists(nodes_path):
    try:
        tmp = gpd.read_file(nodes_path)
        nodes_gdf = tmp[
            tmp.geometry.notnull()
            & ~tmp.geometry.is_empty
            & (tmp.geometry.geom_type == "Point")
        ].copy()
    except Exception as e:
        print(f"Could not read nodes file: {e}")

if (edges_gdf is None or edges_gdf.empty) and os.path.exists(edges_path):
    try:
        tmp = gpd.read_file(edges_path)
        tmp = tmp[tmp.geometry.notnull() & ~tmp.geometry.is_empty].copy()
        edges_gdf = tmp[
            tmp.geometry.geom_type.isin(["LineString", "MultiLineString"])
        ].copy()
    except Exception as e:
        print(f"Could not read edges file: {e}")

if (nodes_gdf is None or nodes_gdf.empty) and (edges_gdf is None or edges_gdf.empty):
    print("Provide a valid combined nodes+edges GeoJSON or separate nodes/edges files.")
else:
    # Determine center/bounds
    center = BLR_CENTER if "BLR_CENTER" in globals() else [12.9716, 77.5946]
    bounds = None
    if edges_gdf is not None and not edges_gdf.empty:
        minx, miny, maxx, maxy = edges_gdf.total_bounds
        bounds = [[miny, minx], [maxy, maxx]]
    if (nodes_gdf is not None and not nodes_gdf.empty) and bounds is None:
        minx, miny, maxx, maxy = nodes_gdf.total_bounds
        bounds = [[miny, minx], [maxy, maxx]]
    if bounds is not None:
        center = [
            (bounds[0][0] + bounds[1][0]) / 2.0,
            (bounds[0][1] + bounds[1][1]) / 2.0,
        ]

    # Use existing map utility if available
    if "create_base_map" in globals():
        m_net = create_base_map(center=center, zoom_start=13)
    else:
        m_net = folium.Map(
            location=center, zoom_start=13, tiles="CartoDB positron", control_scale=True
        )

    # Add edges layer
    if edges_gdf is not None and not edges_gdf.empty:

        def edge_style(_):
            return {"color": "#333", "weight": 2, "opacity": 0.8}

        folium.GeoJson(
            edges_gdf.to_json(), name="Edges", style_function=edge_style
        ).add_to(m_net)

    # Add nodes layer
    if nodes_gdf is not None and not nodes_gdf.empty:
        nodes_fg = folium.FeatureGroup(name="Nodes", show=True)
        for _, r in nodes_gdf.iterrows():
            g = r.geometry
            if g is not None and g.geom_type == "Point":
                folium.CircleMarker(
                    [g.y, g.x],
                    radius=2.5,
                    color="#ff8c00",
                    fill=True,
                    fill_color="#ff8c00",
                    fill_opacity=0.9,
                ).add_to(nodes_fg)
        nodes_fg.add_to(m_net)

    # Fit bounds if we have them
    if bounds is not None:
        try:
            m_net.fit_bounds(bounds)
        except Exception:
            pass

    # Layer control
    if "add_layer_control" in globals():
        add_layer_control(m_net)
    else:
        folium.LayerControl(position="topright").add_to(m_net)

    display(m_net)

### Shortest path on the metro network and final map!

- What this does: Builds a weighted NetworkX graph from the custom nodes/edges GeoJSON, snaps Home and Destination to the nearest nodes, computes the shortest path, and renders the route.
- Outputs: Prints the path summary (stations count and metro line sequence), the walking-distance breakdown, and displays a map with metro lines/stations underneath, the route on top, and dotted walk legs.
- How it works: Edge weights are geodesic distances derived from stored edge geometries; the path is rendered using the original edge shapes where available for accuracy.

In [29]:
# Shortest path on custom nodes/edges network (NetworkX) and put it on the map
import os
import math
import re
import geopandas as gpd
import networkx as nx
import folium
from shapely.geometry import LineString, MultiLineString, Point

# Haversine distance in meters
R = 6371000.0


def haversine_xy(x1, y1, x2, y2):
    lon1, lat1, lon2, lat2 = map(math.radians, [x1, y1, x2, y2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


def geodesic_length(geom) -> float:
    total = 0.0
    if isinstance(geom, LineString):
        coords = list(geom.coords)
        for (x1, y1, *_), (x2, y2, *_) in zip(coords[:-1], coords[1:]):
            total += haversine_xy(x1, y1, x2, y2)
    elif isinstance(geom, MultiLineString):
        for ls in geom.geoms:
            coords = list(ls.coords)
            for (x1, y1, *_), (x2, y2, *_) in zip(coords[:-1], coords[1:]):
                total += haversine_xy(x1, y1, x2, y2)
    return total


def nearest_node_id(nodes_gdf, pt: Point):
    best_id, best_d = None, float("inf")
    for _, r in nodes_gdf.iterrows():
        g = r.geometry
        if g is None or g.is_empty or g.geom_type != "Point":
            continue
        d = haversine_xy(pt.x, pt.y, g.x, g.y)
        if d < best_d:
            best_d = d
            best_id = r.get("node_id") or r.get("id")
    return best_id


def canonical_line_label(edge_data: dict) -> str | None:
    """Return a lowercase color/line label like 'green', 'purple', etc."""
    if not isinstance(edge_data, dict):
        return None
    name = edge_data.get("line_name")
    color = (edge_data.get("line_color") or "").strip()
    # Try extracting color from the name e.g., "Line-2 (Green): ..."
    if isinstance(name, str):
        m = re.search(r"\(([^)]+)\)", name)
        if m:
            return m.group(1).strip().lower()
        # fallback: look for color words in the string
        for w in [
            "green",
            "blue",
            "purple",
            "yellow",
            "pink",
            "red",
            "orange",
            "brown",
        ]:
            if w in name.lower():
                return w
    # Map common hex codes
    hex_map = {
        "#008000": "green",
        "#0000ff": "blue",
        "#800080": "purple",
        "#ffd700": "yellow",
        "#ff69b4": "pink",
        "#ff0000": "red",
        "#ffa500": "orange",
        "#a52a2a": "brown",
    }
    if color:
        return hex_map.get(color.lower())
    return None


# Load combined nodes+edges GeoJSON
combined_path = "metro_network_nodes_edges.geojson"
combo = gpd.read_file(combined_path)
combo = combo[combo.geometry.notnull() & ~combo.geometry.is_empty].copy()

# Split nodes and edges
if "feature_type" in combo.columns:
    nodes_gdf = combo[combo["feature_type"] == "node"].copy()
    edges_gdf = combo[combo["feature_type"] == "edge"].copy()
else:
    nodes_gdf = combo[combo.geometry.geom_type == "Point"].copy()
    edges_gdf = combo[
        combo.geometry.geom_type.isin(["LineString", "MultiLineString"])
    ].copy()

# Build graph
G = nx.Graph()
# Add nodes with coordinates
for _, r in nodes_gdf.iterrows():
    nid = r.get("node_id") or r.get("id")
    g = r.geometry
    if nid and isinstance(g, Point):
        G.add_node(nid, x=g.x, y=g.y, station=r.get("station_name"))

# Map edges to geometries in both directions for easy drawing
edge_geom = {}
for _, r in edges_gdf.iterrows():
    u = r.get("u")
    v = r.get("v")
    geom = r.geometry
    if not u or not v or geom is None:
        continue
    w = geodesic_length(geom) or 0.0
    G.add_edge(
        u,
        v,
        weight=w,
        edge_id=r.get("edge_id"),
        line_name=r.get("line_name"),
        line_color=r.get("line_color"),
    )
    # store coords as lat-lon for folium
    if isinstance(geom, (LineString, MultiLineString)):
        if isinstance(geom, LineString):
            coords = [(lat, lon) for lon, lat, *rest in list(geom.coords)]
        else:
            # flatten multilines into a single sequence for drawing
            coords = []
            for ls in geom.geoms:
                coords += [(lat, lon) for lon, lat, *rest in list(ls.coords)]
        edge_geom[(u, v)] = coords
        edge_geom[(v, u)] = list(reversed(coords))

# Determine start and end Points
sp = globals().get("start_point", None)
if sp is None and "home_gdf" in globals() and len(home_gdf) > 0:
    try:
        sp = home_gdf.geometry.iloc[0]
    except Exception:
        sp = None

ep = globals().get("endpoint_point", None)
if ep is None and "restaurants_gdf" in globals() and len(restaurants_gdf) > 0:
    # fallback to first restaurant
    try:
        ep = restaurants_gdf.geometry.iloc[0]
    except Exception:
        ep = None

if sp is None or ep is None:
    raise RuntimeError(
        "Start or end point is not set. Pick a destination in the dropdown cell above and re-run, or ensure home/restaurant data is loaded."
    )

# Snap to nearest nodes in the graph
src_id = nearest_node_id(nodes_gdf, sp)
dst_id = nearest_node_id(nodes_gdf, ep)

if src_id is None or dst_id is None:
    raise RuntimeError("Could not snap start or end to a network node.")

# Compute shortest path
try:
    path = nx.shortest_path(G, src_id, dst_id, weight="weight")
except nx.NetworkXNoPath:
    raise RuntimeError(f"No path between {src_id} and {dst_id} in the current graph.")

# Total length and line changes summary
dist_m = 0.0
edge_line_labels = []
for u, v in zip(path[:-1], path[1:]):
    w = G[u][v].get("weight", 0.0)
    dist_m += w
    edge_line_labels.append(canonical_line_label(G[u][v]))

# Collapse consecutive duplicate line labels and drop Nones
line_sequence = []
for lab in edge_line_labels:
    if not lab:
        continue
    if not line_sequence or lab != line_sequence[-1]:
        line_sequence.append(lab)

if line_sequence:
    print(
        f"Shortest path: {len(path)} stations, {dist_m/1000:.2f} km — lines: "
        + " to ".join(line_sequence)
    )
else:
    print(f"Shortest path: {len(path)} stations, {dist_m/1000:.2f} km")

# Compute and print walking distance breakdown
walk_start_m = walk_end_m = None
try:
    g_start = nodes_gdf[nodes_gdf["node_id"] == src_id].geometry.iloc[0]
    if sp is not None:
        walk_start_m = haversine_xy(sp.x, sp.y, g_start.x, g_start.y)
except Exception:
    walk_start_m = None
try:
    g_end = nodes_gdf[nodes_gdf["node_id"] == dst_id].geometry.iloc[0]
    if ep is not None:
        walk_end_m = haversine_xy(ep.x, ep.y, g_end.x, g_end.y)
except Exception:
    walk_end_m = None

total_walk_m = (walk_start_m or 0.0) + (walk_end_m or 0.0)
if walk_start_m is not None:
    print(
        f"Walk: Home → first station: {walk_start_m:.0f} m ({walk_start_m/1000:.2f} km)"
    )
else:
    print("Walk: Home → first station: n/a")
if walk_end_m is not None:
    print(
        f"Walk: last station → destination: {walk_end_m:.0f} m ({walk_end_m/1000:.2f} km)"
    )
else:
    print("Walk: last station → destination: n/a")
print(f"Total walking distance: {total_walk_m:.0f} m ({total_walk_m/1000:.2f} km)")

# Build the map
center = [(sp.y + ep.y) / 2.0, (sp.x + ep.x) / 2.0]
if "create_base_map" in globals():
    m = create_base_map(center=center, zoom_start=12)
else:
    m = folium.Map(
        location=center, zoom_start=12, tiles="CartoDB positron", control_scale=True
    )

# Add metro lines and stations UNDER the route (add these first)
try:
    # Try to use preloaded GeoDataFrames if available
    lines_gdf = globals().get("metro_lines_gdf", None)
    stations_gdf = globals().get("metro_stations_gdf", None)
    if (lines_gdf is None or lines_gdf.empty) or (
        stations_gdf is None or stations_gdf.empty
    ):
        # Fallback: read and split from source file
        metro_path = "metro_lines_styled_cgpt.geojson"
        _all = gpd.read_file(metro_path)
        _all = _all[_all.geometry.notnull() & ~_all.geometry.is_empty].copy()
        _gt = _all.geometry.geom_type
        lines_gdf = _all[_gt.isin(["LineString", "MultiLineString"])].copy()
        stations_gdf = _all[_gt.isin(["Point", "MultiPoint"])].copy()
        if (stations_gdf.geometry.geom_type == "MultiPoint").any():
            stations_gdf = stations_gdf.explode(index_parts=False, ignore_index=True)
    # Use helpers if present, otherwise simple add
    if "add_metro_lines_layer" in globals():
        add_metro_lines_layer(m, lines_gdf, name="Metro Lines")
    else:
        folium.GeoJson(
            lines_gdf.to_json(),
            name="Metro Lines",
            style_function=lambda f: {"color": "#888", "weight": 2, "opacity": 0.7},
        ).add_to(m)
    if "add_metro_stations_layer" in globals():
        add_metro_stations_layer(m, stations_gdf, name="Metro Stations")
    else:
        st_fg = folium.FeatureGroup(name="Metro Stations", show=True)
        for _, r in stations_gdf.iterrows():
            g = r.geometry
            if g is not None and g.geom_type == "Point":
                folium.CircleMarker(
                    [g.y, g.x],
                    radius=4,
                    color="#000",
                    fill=True,
                    fill_color="#fff",
                    fill_opacity=0.9,
                ).add_to(st_fg)
        st_fg.add_to(m)
except Exception as e:
    print(f"Could not add metro lines/stations: {e}")

# Add Home and Destination markers
folium.Marker(
    [sp.y, sp.x],
    icon=folium.Icon(color="blue", icon="home", prefix="fa"),
    tooltip="Home",
).add_to(m)
folium.Marker(
    [ep.y, ep.x],
    icon=folium.Icon(color="red", icon="map-marker", prefix="fa"),
    tooltip=globals().get("endpoint_name", "Destination"),
).add_to(m)

# Add snapped node highlights
snaps = folium.FeatureGroup(name="Snapped Nodes", show=True)
ns = (
    nodes_gdf.set_index(nodes_gdf["node_id"])
    if "node_id" in nodes_gdf.columns
    else None
)
for nid, color, label in [
    (src_id, "#2ecc71", "Start node"),
    (dst_id, "#e74c3c", "End node"),
]:
    try:
        row = nodes_gdf[nodes_gdf["node_id"] == nid].iloc[0]
        g = row.geometry
        folium.CircleMarker(
            [g.y, g.x],
            radius=6,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.95,
            tooltip=f"{label}: {nid}",
        ).add_to(snaps)
    except Exception:
        pass
snaps.add_to(m)

# Dotted walk legs from points to snaps
walk = folium.FeatureGroup(name="Walk legs", show=True)
try:
    g_start = nodes_gdf[nodes_gdf["node_id"] == src_id].geometry.iloc[0]
    folium.PolyLine(
        [[sp.y, sp.x], [g_start.y, g_start.x]], color="#666", weight=3, dash_array="6,8"
    ).add_to(walk)
except Exception:
    pass
try:
    g_end = nodes_gdf[nodes_gdf["node_id"] == dst_id].geometry.iloc[0]
    folium.PolyLine(
        [[g_end.y, g_end.x], [ep.y, ep.x]], color="#666", weight=3, dash_array="6,8"
    ).add_to(walk)
except Exception:
    pass
walk.add_to(m)

# Draw the route as concatenated edge geometries (added LAST so it stays on top)
route_fg = folium.FeatureGroup(name="Route (shortest path)", show=True)
for u, v in zip(path[:-1], path[1:]):
    coords = edge_geom.get((u, v))
    if not coords:
        # fallback to straight segment between node coords
        try:
            ru = nodes_gdf[nodes_gdf["node_id"] == u].geometry.iloc[0]
            rv = nodes_gdf[nodes_gdf["node_id"] == v].geometry.iloc[0]
            coords = [[ru.y, ru.x], [rv.y, rv.x]]
        except Exception:
            coords = None
    if coords:
        folium.PolyLine(coords, color="#ff7f0e", weight=6, opacity=0.9).add_to(route_fg)
route_fg.add_to(m)

# Optionally add layer control
if "add_layer_control" in globals():
    add_layer_control(m)
else:
    folium.LayerControl(position="topright").add_to(m)

m

Shortest path: 10 stations, 10.04 km — lines: green to yellow
Walk: Home → first station: 480 m (0.48 km)
Walk: last station → destination: 1877 m (1.88 km)
Total walking distance: 2357 m (2.36 km)


### We have found the fastest route to get to your favorite restaurant through Namma Metro!